In [ ]:
usersPath = 'ExampleData/Users.txt'
appsPath = 'ExampleData/Apps.txt'
actionPath = 'ExampleData/Actions.txt'
output1 = 'outPart1/'
output2 = 'outPart2/'

In [ ]:
### PART 1
actions_rdd = sc.textFile(actionPath).cache()

In [ ]:
#Select install and the year 2022
def Install2022(line):
    fields = line.split(',')
    timestamp = fields[2]
    action = fields[3]
    
    return timestamp.startswith('2022')==True and action=='Install'
    

actions2022_rdd = actions_rdd.filter(Install2022)

In [ ]:
# Map to (appId, userId) and
# remove repeated pairs

def appUser(line):
    fields = line.split(',')
    userId = fields[0]
    appId = fields[1]
    
    return (appId, userId)


appUser_rdd = actions2022_rdd.map(appUser)\
.distinct()

In [ ]:
apps_rdd = sc.textFile(appsPath)

In [ ]:
# Map apps to to (appId, Price)
def appPrice(line):
    fields = line.split(',')
    appId = fields[0]
    price = float(fields[2])
    
    return (appId, price)


appPrice_rdd = apps_rdd.map(appPrice)

In [ ]:
# Join appPrice_rdd with appUser_rdd
joinActionsPrices = appPrice_rdd.join(appUser_rdd)

In [ ]:
# Map each pair to (userId, (free , non-free))
def freeNonfree(pair):
    price=pair[1][0]
    userId=pair[1][1]

    if (price>0):
        return (userId, (0, 1))
    else:
        return (userId, (1, 0))
    

userFreeNonfreeApps_rdd = joinActionsPrices.map(freeNonfree)

In [ ]:
# Count the number of free and non-free apps for each user.
# Select the users with more non-free apps than free apps

selectedUsersPart1_rdd = userFreeNonfreeApps_rdd.reduceByKey(lambda v1,v2: (v1[0]+v2[0], v1[1]+v2[1]))\
.filter(lambda pair: pair[1][1]>pair[1][0])

In [ ]:
# Keep only userId and number of non-free apps and store the result
result1 = selectedUsersPart1_rdd.mapValues(lambda v: v[1])

In [ ]:
#result1.collect()

In [ ]:
result1.saveAsTextFile(output1)

In [ ]:
### PART 2
users_rdd = sc.textFile(usersPath)

In [ ]:
# Select Italian users and return pairs (UserId, None) 
italianUsers_rdd = users_rdd.filter(lambda line: line.split(',')[3]=='Italian')\
.map(lambda line: (line.split(',')[0], None))

In [ ]:
# Map each action to ( userId (appId,timestamp, action))
def  extractPair(line):
    fields = line.split(',')
    userId = fields[0]
    appId = fields[1]
    timestamp = fields[2]
    action = fields[3]
    
    return ( (userId, (appId, timestamp, action)) )
    
    
usersActions_rdd = actions_rdd.map(extractPair)

In [ ]:
# Join usersActions_rdd with italianUsers_rdd
# and select only Italian users
italianUsersActions_rdd  = usersActions_rdd.join(italianUsers_rdd)

In [ ]:
#italianUsersActions_rdd.collect()

In [ ]:
# Compute the last action for each combination Italian user+app
# Map to ( (userId, appId) , (timestamp, action))
# and then compute max timestamp + action

def maxTimestamp(time1_action1, time2_action2):
    if (time1_action1[0]>time2_action2[0]):
        return time1_action1
    else:
        return time2_action2    

usersLasAction_rdd = italianUsersActions_rdd.map(lambda p: ( (p[0], p[1][0][0]), (p[1][0][1], p[1][0][2]) ) )\
.reduceByKey(maxTimestamp)

In [ ]:
#usersLasAction_rdd.collect()

In [ ]:
# Select the apps with last action equal to Install => currently installed app
users_installedapps = usersLasAction_rdd.filter(lambda p: p[1][1]=='Install')

In [ ]:
# Count the number of currently installed apps for each user
usersNumInstalledapps = users_installedapps.map(lambda p: (p[0][0], +1))\
.reduceByKey(lambda v1,v2: v1+v2).cache()

In [ ]:
# Compute the maximum value of number of installed apps
maxInstalledApps = usersNumInstalledapps.values().max()

In [ ]:
# Select the users associated with maxInstalledApps
result2 = usersNumInstalledapps.filter(lambda p: p[1]==maxInstalledApps)\
.keys()

In [ ]:
#result2.collect()

In [ ]:
result2.saveAsTextFile(output2)